## MODELADO DE DATOS CON BIGQUERY ML - Regresión Lineal

En esta notebook se calcula una regresión lineal para una predicción.
Se carga una base se datos, se entrena el modelo, se lo evalua y se predicen nuevos datos.
Ademas se trabaja con la explicación a nivel fila y global, para entender que varias son relevantes.

In [4]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


Creo una tabla para modelar

Creo el modelo 

In [1]:
%%bigquery

# Creating the baseline model
CREATE OR REPLACE MODEL
  `proyectos.nps.bigquery_ml_wei_lineal_reg`
    OPTIONS (model_type='linear_reg', input_label_cols=['nps_numero']
   ,num_trials=5 ##HIPERPARAMETRO
   ,max_parallel_trials=3, ##HIPERPARAMETRO
   l1_reg=hparam_range(0, 1), ##HIPERPARAMETRO
   l2_reg=hparam_candidates([0, 0.1, 1, 10]) ##HIPERPARAMETRO
            ) AS
SELECT
  wtod_score_modem_avg, wtod_score_modem_stddev, wtod_score_modem_min,  wtod_score_modem_max,
    wbhp_score_modem_avg, wbhp_score_modem_stddev,  wbhp_score_modem_min,  wbhp_score_modem_max,
    wts_score_modem_avg, wts_score_modem_stddev, wts_score_modem_min, wts_score_modem_max,
    wdr_score_modem_avg, wdr_score_modem_stddev,  wdr_score_modem_min,  wdr_score_modem_max,
    wpl_score_modem_avg, wpl_score_modem_stddev, wpl_score_modem_min, wpl_score_modem_max,
    wi_score_modem_avg, wi_score_modem_stddev,  wi_score_modem_min,  wi_score_modem_max,
    wlbu_score_modem_avg, wlbu_score_modem_stddev, wlbu_score_modem_min, wlbu_score_modem_max, 
    ML.FEATURE_CROSS(STRUCT(modem, tecnologia)) AS modem_tecnologia, ## le indico dos variables y me calcula las combinaciones
    ML.POLYNOMIAL_EXPAND(STRUCT(wi_score_modem_avg), 3) as wi_score_polinomio, ##creates x, x2, x3, etc.
    ML.BUCKETIZE(wi_score_modem_max, [20, 40, 60, 80]) as wi_score_bucket, ## where split_points is an array
   nps_numero 
FROM
  `proyectos.nps.qualtrics_wei_train_data`
WHERE 
    ABS(MOD(FARM_FINGERPRINT(TO_JSON_STRING(STRUCT(dt, contrato, modem))), 10)) < 8 ##SPLIT Y ENTRENO
    


Query complete after 0.01s: 100%|██████████| 11/11 [00:00<00:00, 5631.31query/s]                       


""


Obtener las estadisticas de entrenamiento

In [15]:
%%bigquery
SELECT
  *
FROM
  --ML.TRAINING_INFO(MODEL `proyectos.nps.bigquery_ml_wei_lineal_reg`)    #SIN TUNEO HIPERPARAMETROS
  ML.TRIAL_INFO(MODEL `proyectos.nps.bigquery_ml_wei_lineal_reg`)     #SOLO SI TUNEO HIPERPARAMETROS


Downloading: 100%|██████████| 5/5 [00:00<00:00,  6.71rows/s]


,trial_id,hyperparameters,hparam_tuning_evaluation_metrics,training_loss,eval_loss,status,error_message,is_optimal
0,1,"{'l1_reg': 1e-14, 'l2_reg': 1.0}",{'r2_score': 0.019128155849764794},10.979022,11.024662,SUCCEEDED,None,False
1,2,"{'l1_reg': 1.1924052757445675e-11, 'l2_reg': 1.0}",{'r2_score': 0.019128155849765127},10.979022,11.024662,SUCCEEDED,None,True
2,3,"{'l1_reg': 2.6958670578562742e-08, 'l2_reg': 0.1}",{'r2_score': 0.01512969937773534},10.965046,11.069603,SUCCEEDED,None,False
3,4,"{'l1_reg': 2.0403139161274637e-10, 'l2_reg': 1.0}",{'r2_score': 0.01912815584976324},10.979022,11.024662,SUCCEEDED,None,False
4,5,"{'l1_reg': 2.0728085351859456e-12, 'l2_reg': 1.0}",{'r2_score': 0.019128155849765127},10.979022,11.024662,SUCCEEDED,None,True


# Evaluación del modelo

In [2]:
%%bigquery
# Evaluate Model
SELECT *
FROM
  ML.EVALUATE(MODEL `proyectos.nps.bigquery_ml_wei_lineal_reg`,
    (SELECT *, 
     ML.FEATURE_CROSS(STRUCT(modem, tecnologia)) AS modem_tecnologia,
     ML.POLYNOMIAL_EXPAND(STRUCT(wi_score_modem_avg), 3) as wi_score_polinomio, ##creates x, x2, x3, etc.
     ML.BUCKETIZE(wi_score_modem_max, [20, 40, 60, 80]) as wi_score_bucket, ## where split_points is an array   ##SI AGREGO ALGUNA VARIABLE LA DEBO METER ACA TAMBIEN 
     FROM `proyectos.nps.qualtrics_wei_train_data` 
     WHERE ABS(MOD(FARM_FINGERPRINT(TO_JSON_STRING(STRUCT(dt, contrato, modem))), 10)) >= 8 ##SPLIT Y ENTRENO
    ))


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.26rows/s]


,trial_id,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,5,2.929214,11.631282,0.7284,2.967536,-0.000078,-0.000076


In [3]:
%%bigquery

# Use `ML.EVALUATE` function to evaluate model metrics. 
SELECT
  SQRT(mean_squared_error) AS rmse
FROM
  ML.EVALUATE(MODEL `proyectos.nps.bigquery_ml_wei_lineal_reg`)
    

Downloading: 100%|██████████| 5/5 [00:00<00:00,  6.29rows/s]


,rmse
0,3.183858
1,3.183497
2,3.196667
3,3.196667
4,3.196667


predicción

In [4]:
%%bigquery

# Predict
SELECT
  *
FROM
  ML.PREDICT(MODEL `proyectos.nps.bigquery_ml_wei_lineal_reg`,
    (SELECT FARM_FINGERPRINT(CONCAT(CAST(tecnologia AS STRING), wtod_score_modem_min, CAST(modem AS STRING))) as row_fingerprint,
rand() as random, modem, tecnologia,
  wtod_score_modem_avg, wtod_score_modem_stddev, wtod_score_modem_min,  wtod_score_modem_max,
    wbhp_score_modem_avg, wbhp_score_modem_stddev,  wbhp_score_modem_min,  wbhp_score_modem_max,
    wts_score_modem_avg, wts_score_modem_stddev, wts_score_modem_min, wts_score_modem_max,
    wdr_score_modem_avg, wdr_score_modem_stddev,  wdr_score_modem_min,  wdr_score_modem_max,
    wpl_score_modem_avg, wpl_score_modem_stddev, wpl_score_modem_min, wpl_score_modem_max,
    wi_score_modem_avg, wi_score_modem_stddev,  wi_score_modem_min,  wi_score_modem_max,
    wlbu_score_modem_avg, wlbu_score_modem_stddev, wlbu_score_modem_min, wlbu_score_modem_max, 
     ML.FEATURE_CROSS(STRUCT(modem, tecnologia)) AS modem_tecnologia,
     ML.POLYNOMIAL_EXPAND(STRUCT(wi_score_modem_avg), 3) as wi_score_polinomio, ##creates x, x2, x3, etc.
     ML.BUCKETIZE(wi_score_modem_max, [20, 40, 60, 80]) as wi_score_bucket,
     FROM `proyectos.nps.qualtrics_wei_train_data` limit 1000));


Downloading: 100%|██████████| 1000/1000 [00:00<00:00, 1241.39rows/s]


,trial_id,predicted_nps_numero,row_fingerprint,random,modem,tecnologia,wtod_score_modem_avg,wtod_score_modem_stddev,wtod_score_modem_min,wtod_score_modem_max,...,wi_score_modem_stddev,wi_score_modem_min,wi_score_modem_max,wlbu_score_modem_avg,wlbu_score_modem_stddev,wlbu_score_modem_min,wlbu_score_modem_max,modem_tecnologia,wi_score_polinomio,wi_score_bucket
0,5,5.073715,4484183361832963093,0.854155,tchn4233,HFC,0.000000,0.000000,0.00,0.000,...,5.398171,42.101187,53.237585,0.000000,0.000000,0.00,0.00,{'modem_tecnologia': 'tchn4233_HFC'},"{'wi_score_modem_avg': 48.568663455187604, 'wi...",bin_3
1,5,5.100909,-5339490955989020034,0.948686,smbs3686,HFC,0.000000,0.000000,0.00,0.000,...,0.204888,15.923567,16.543624,100.000000,0.000000,100.00,100.00,{'modem_tecnologia': 'smbs3686_HFC'},"{'wi_score_modem_avg': 16.221653628443587, 'wi...",bin_1
2,5,5.384797,6085560282673740129,0.197126,smbs3890,HFC,0.000000,0.000000,0.00,0.000,...,0.000000,0.000000,0.000000,67.924286,1.831128,65.56,70.70,{'modem_tecnologia': 'smbs3890_HFC'},"{'wi_score_modem_avg': 0.0, 'wi_score_modem_av...",bin_1
3,5,5.032930,6085560282673740129,0.086771,smbs3890,HFC,0.000000,0.000000,0.00,0.000,...,0.000000,0.000000,0.000000,98.961429,0.392998,98.63,99.74,{'modem_tecnologia': 'smbs3890_HFC'},"{'wi_score_modem_avg': 0.0, 'wi_score_modem_av...",bin_1
4,5,4.974768,4349173671011907940,0.320301,tchn3848,HFC,0.000000,0.000000,0.00,0.000,...,0.000000,0.000000,0.000000,43.292857,7.547385,32.69,53.30,{'modem_tecnologia': 'tchn3848_HFC'},"{'wi_score_modem_avg': 0.0, 'wi_score_modem_av...",bin_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5,5.797156,-6530293540636871171,0.532519,smbs5657,FTTH,0.129286,0.236651,0.00,0.600,...,0.374615,24.192073,25.375767,0.000000,0.000000,0.00,0.00,{'modem_tecnologia': 'smbs5657_FTTH'},"{'wi_score_modem_avg': 24.784317704664623, 'wi...",bin_2
996,5,5.804819,2916557903978646332,0.454380,smbs5657,FTTH,37.041429,2.639152,34.11,41.610,...,0.481464,35.203125,36.608116,0.000000,0.000000,0.00,0.00,{'modem_tecnologia': 'smbs5657_FTTH'},"{'wi_score_modem_avg': 36.10663608954553, 'wi_...",bin_2
997,5,5.939232,-6530293540636871171,0.299574,smbs5657,FTTH,12.214286,8.674285,0.00,20.360,...,0.311845,33.795732,34.655689,0.000000,0.000000,0.00,0.00,{'modem_tecnologia': 'smbs5657_FTTH'},"{'wi_score_modem_avg': 34.31109845877745, 'wi_...",bin_2
998,5,5.903946,-7128826710360303480,0.601135,smbs5657,FTTH,22.220714,3.565626,18.18,26.345,...,0.195382,32.705793,33.277439,0.000000,0.000000,0.00,0.00,{'modem_tecnologia': 'smbs5657_FTTH'},"{'wi_score_modem_avg': 32.9792731985244, 'wi_s...",bin_2


## Explicar el modelo a nivel fila

In [89]:
%%bigquery

#Explain Predictions
SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL `teco-prod-wei-dev-29cd.nps.bigquery_ml_wei_lineal_reg`,
    (SELECT *, 
     ML.FEATURE_CROSS(STRUCT(modem, tecnologia)) AS modem_tecnologia,
     ML.POLYNOMIAL_EXPAND(STRUCT(wi_score_modem_avg), 3) as wi_score_polinomio, ##creates x, x2, x3, etc.
     ML.BUCKETIZE(wi_score_modem_max, [20, 40, 60, 80]) as wi_score_bucket,
     FROM `teco-prod-wei-dev-29cd.nps.qualtrics_wei_train_data` limit 1000),
    STRUCT(3 as top_k_features));

Downloading: 100%|██████████| 1000/1000 [00:01<00:00, 792.02rows/s] 


,trial_id,predicted_nps,probability,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,contrato,tecnologia,modem,...,wi_score_modem_avg,wlbu_score_modem_avg,wtod_score_modem_min,wbhp_score_modem_min,wts_score_modem_min,wdr_score_modem_min,wpl_score_modem_min,wi_score_modem_min,wlbu_score_modem_min,modem_tecnologia
0,1,0,0.559779,[{'feature': 'modem_tecnologia_modem_tecnologi...,0.051289,-0.240266,0.0,13045315,HFC,kaon3000,...,0.000000,0.000000,0.0,10.000000,0.000,-5.603301e-05,NaN,0.000000,0.00,{'modem_tecnologia': 'kaon3000_HFC'}
1,1,0,0.560045,[{'feature': 'modem_tecnologia_modem_tecnologi...,0.051289,-0.241343,0.0,12986951,HFC,kaon3000,...,0.000000,0.000000,0.0,0.000000,0.000,-1.018172e-06,NaN,0.000000,0.00,{'modem_tecnologia': 'kaon3000_HFC'}
2,1,0,0.559274,[{'feature': 'modem_tecnologia_modem_tecnologi...,0.051289,-0.238214,0.0,38589740,HFC,kaon3000,...,0.000000,0.000000,0.0,28.000000,0.000,-1.040834e-15,NaN,0.000000,0.00,{'modem_tecnologia': 'kaon3000_HFC'}
3,1,1,0.531060,[{'feature': 'modem_tecnologia_modem_tecnologi...,0.051289,0.124398,0.0,13102422,HFC,smbs3686,...,43.613351,100.000000,0.0,10.000000,93.020,2.400020e+00,60.000000,42.448276,100.00,{'modem_tecnologia': 'smbs3686_HFC'}
4,1,1,0.534394,[{'feature': 'modem_tecnologia_modem_tecnologi...,0.051289,0.137792,0.0,79147178,HFC,smbs3686,...,16.608500,71.226250,0.0,46.666667,23.460,4.865487e+00,72.280000,16.336276,67.60,{'modem_tecnologia': 'smbs3686_HFC'}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,1,0.525311,[{'feature': 'modem_tecnologia_modem_tecnologi...,0.051289,0.101329,0.0,89809987,HFC,tchn3848,...,0.000000,96.135000,0.0,17.000000,5.960,7.745482e-01,40.000000,0.000000,92.79,{'modem_tecnologia': 'tchn3848_HFC'}
996,1,1,0.525121,[{'feature': 'modem_tecnologia_modem_tecnologi...,0.051289,0.100567,0.0,60632404,HFC,tchn3848,...,0.000000,100.000000,0.0,1.666667,14.250,1.000001e+00,77.003333,0.000000,100.00,{'modem_tecnologia': 'tchn3848_HFC'}
997,1,1,0.525073,[{'feature': 'modem_tecnologia_modem_tecnologi...,0.051289,0.100377,0.0,39953395,HFC,tchn3848,...,0.000000,11.821429,0.0,0.000000,5.755,5.102041e-01,0.000000,0.000000,6.41,{'modem_tecnologia': 'tchn3848_HFC'}
998,1,1,0.525791,[{'feature': 'modem_tecnologia_modem_tecnologi...,0.051289,0.103257,0.0,12932586,HFC,tchn3848,...,0.000000,95.750000,0.0,22.727273,9.720,3.999911e-01,65.054545,0.000000,94.81,{'modem_tecnologia': 'tchn3848_HFC'}
